!jt -t chesterish
# !jt -r

In [1]:
from genetic_algorithm import *
from numpy.random import default_rng
from tqdm import tqdm
from itertools import product #combinations_with_replacement
%matplotlib notebook

import matplotlib.pyplot as plt
import math
from matplotlib.animation import FuncAnimation

import copy
from collections import Counter
import pandas as pd

In [2]:
## hexadecimal generator

nr_of_input = 2
nr_of_actions = 5
nr_of_inner = 3
nr_of_genes = 8
nr_individuals = 1200

individuals = generate_initial_genomes_for_population(nr_individuals, nr_of_genes, nr_of_input, nr_of_actions, nr_of_inner)

## world size
ws = 128
world_size_x = ws
world_size_y = ws

##TODo if world_size_x != world_size_y fit then limit coord_full_list 

coord_full_list = np.array(list(product(range(ws), repeat=2)))
ind = np.random.choice(len(coord_full_list), nr_individuals, replace=False)
pos = coord_full_list[ind]

## initial brain and position generator

result = calculate_individual_output_weights(individuals)

## assign position and remove brains without output

indiv_to_del = []
for indiv in result:
    if result[indiv]['out']:
        result[indiv]['position'] = [list(pos[indiv])]
    else:
        indiv_to_del.append(indiv)

for key in indiv_to_del:
    del result[key] 

In [3]:
# rr = copy.copy(result)
# rr

In [ ]:
nr_of_errors = 0

k = 120
n = 0
pbar = tqdm(total=k, initial=n)

while k>n: 
    pbar.update(1)
    for indiv in result:
        x, y = result[indiv]['position'][-1][0], result[indiv]['position'][-1][1]
        if n<1:
            calculate_position(result, indiv, x, y, world_size_x, world_size_y)    
        elif n >= 1:
            apply_input(result, indiv)
            calculate_position(result, indiv, x, y, world_size_x, world_size_y)
#     if n>1:
    last_pos_list = {obj:result[obj]['position'][-1] for obj in result}
#     print(n)
#     print(last_pos_list)
#     print()
    for key_1, val_1 in last_pos_list.items():
        last_pos_list_copy = copy.copy(last_pos_list)
        del last_pos_list_copy[key_1]
        for key_2, val_2 in last_pos_list_copy.items():
            if val_1 == val_2:
#                 print(key_1, val_1, key_2, val_2)
#                 print(result[key_1]['position'][-1], result[key_1]['position'][-2])
                result[key_2]['position'][-1] = result[key_2]['position'][-2]
                result[key_1]['position'][-1] = result[key_1]['position'][-2]
#     else:
#         last_pos_list = {obj:result[obj]['position'][-1] for obj in result}
    n += 1
pbar.close()


 30%|█████████▉                       | 36/120 [00:11<00:29,  2.83it/s]

In [ ]:
result

In [ ]:
# position_list = list(map(sum, zip(*[[0, -1], [1, 0], [0, 0], [0, 1]])))
# print(position_list)
# position_list = make_smaller_(position_list)
# print(position_list, x,y)
# position_list = list(map(sum, zip(*[[x, y]] + [position_list])))
# print(position_list)

In [ ]:
# result[key_1]['position'][-1]

In [ ]:
df = pd.DataFrame()
for key in result:
    df[key] = pd.Series(result[key]['position']).apply(lambda x: '.'.join((str(x[0]),str(x[1]))))

print(Counter(df.iloc[-1,:]).most_common())

df

In [ ]:
df.iloc[0,:].head(30)


In [ ]:
# last_pos_list= [[tuple(result[obj]['position'][-1])] for obj in result]

In [ ]:
# from itertools import chain
# Counter(chain(* l)).most_common()

generate dictionary of coords 

In [ ]:
coords = [{'x':[], 'y':[]} for key in range(len(result[indiv]['position']))]
for indiv in result:
    for pos_nr, pos in enumerate(result[indiv]['position']):
        coords[pos_nr]['x'].append(pos[0])
        coords[pos_nr]['y'].append(pos[1])

visualisation

In [ ]:
def update(i):
    ax.clear()
    ax.set_facecolor(plt.cm.Blues(.2))

    ax.set_xlim([0,world_size_x])
    ax.set_ylim([0,world_size_y])
    ax.set_title('moving')
    ax.scatter(x=coords[i]['x'],y=coords[i]['y'], c='red', s=2, marker='o')
    [spine.set_visible(False) for spine in ax.spines.values()] #remove chart 
    
fig, ax = plt.subplots(figsize=(6,6))
anime = FuncAnimation(
    fig = fig,
    func = update,
    frames = len(coords),
    interval = 10, repeat=False
)


### mutation

hexa to binary

In [14]:
binary = bin(int(individuals[0][0].hex_id, 16))[2:]
if len(binary) < 32:
    factor = 32 - len(binary)
    binary = '0' * factor + binary
binary

'00011010001100001101001001111100'